In [1]:
import pandas as pd 
import numpy as np 
import json, time 
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertConfig, BertTokenizer, AdamW, get_cosine_schedule_with_warmup
import warnings
warnings.filterwarnings('ignore')

In [2]:
model_name = 'hfl/chinese-roberta-wwm-ext-large'
#model_name = 'hfl/chinese-roberta-wwm-ext'     #换用模型
#model_name = 'hfl/chinese-bert-wwm-ext'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
sen = 'Transformers提供了NLP领域大量state-of-art的 预训练语言模型结构的模型和调用框架。'
inputs = tokenizer(sen, return_tensors='pt')
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(inputs)
print(tokens)
outputs = model(**inputs)
print(len(outputs))
print(outputs[0].shape, outputs[1].shape)

{'input_ids': tensor([[  101,   162, 10477,  8118, 12725,  8755,  2990,   897,   749,   156,
         10986,  7566,  1818,  1920,  7030, 10223,   118,  8205,   118,  9143,
          4638,  7564,  6378,  5298,  6427,  6241,  3563,  1798,  5310,  3354,
          4638,  3563,  1798,  1469,  6444,  4500,  3427,  3373,   511,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['[CLS]', 't', '##ran', '##s', '##form', '##ers', '提', '供', '了', 'n', '##lp', '领', '域', '大', '量', 'state', '-', 'of', '-', 'art', '的', '预', '训', '练', '语', '言', '模', '型', '结', '构', '的', '模', '型', '和', '调', '用', '框', '架', '。', '[SEP]']
2
torch.Size([1, 40, 1024]) torch.Size([1, 1024])


In [4]:
import unicodedata

data_nums = []
maxlen = 100      

train_input_ids, train_input_masks, train_input_types, train_labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
valid_input_ids, valid_input_masks, valid_input_types, valid_labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
test_input_ids, test_input_masks, test_input_types, test_labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask

with open("./data/news/体育.txt", encoding='utf-8') as f:
    input_ids, input_masks, input_types, labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
    for i, line in tqdm(enumerate(f)): 
        content = unicodedata.normalize('NFKC', line.strip())
        # encode_plus会输出一个字典，分别为'input_ids', 'token_type_ids', 'attention_mask'对应的编码
        # 根据参数会短则补齐，长则切断
        encode_dict = tokenizer.encode_plus(text=content, max_length = maxlen, 
                                            padding='max_length', truncation=True)
        input_ids.append(encode_dict['input_ids'])
        input_types.append(encode_dict['token_type_ids'])
        input_masks.append(encode_dict['attention_mask'])

        labels.append([1,0,0,0,0])

    train_input_ids.extend(np.array(input_ids[:int(i*0.8)]))
    train_input_types.extend(np.array(input_types[:int(i*0.8)]))
    train_input_masks.extend(np.array(input_masks[:int(i*0.8)]))
    train_labels.extend(np.array(labels[:int(i*0.8)]))    
    valid_input_ids.extend(np.array(input_ids[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_types.extend(np.array(input_types[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_masks.extend(np.array(input_masks[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_labels.extend(np.array(labels[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    test_input_ids.extend(np.array(input_ids[int(i*0.8)+int(i*0.1):]))
    test_input_types.extend(np.array(input_types[int(i*0.8)+int(i*0.1):]))
    test_input_masks.extend(np.array(input_masks[int(i*0.8)+int(i*0.1):]))
    test_labels.extend(np.array(labels[int(i*0.8)+int(i*0.1):]))
    data_nums.append(i+1)

with open("./data/news/娱乐.txt", encoding='utf-8') as f:
    input_ids, input_masks, input_types, labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
    for i, line in tqdm(enumerate(f)): 
        content = unicodedata.normalize('NFKC', line.strip())
        # encode_plus会输出一个字典，分别为'input_ids', 'token_type_ids', 'attention_mask'对应的编码
        # 根据参数会短则补齐，长则切断
        encode_dict = tokenizer.encode_plus(text=content, max_length = maxlen, 
                                            padding='max_length', truncation=True)
        input_ids.append(encode_dict['input_ids'])
        input_types.append(encode_dict['token_type_ids'])
        input_masks.append(encode_dict['attention_mask'])

        labels.append([0,1,0,0,0])

    train_input_ids.extend(np.array(input_ids[:int(i*0.8)]))
    train_input_types.extend(np.array(input_types[:int(i*0.8)]))
    train_input_masks.extend(np.array(input_masks[:int(i*0.8)]))
    train_labels.extend(np.array(labels[:int(i*0.8)]))    
    valid_input_ids.extend(np.array(input_ids[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_types.extend(np.array(input_types[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_masks.extend(np.array(input_masks[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_labels.extend(np.array(labels[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    test_input_ids.extend(np.array(input_ids[int(i*0.8)+int(i*0.1):]))
    test_input_types.extend(np.array(input_types[int(i*0.8)+int(i*0.1):]))
    test_input_masks.extend(np.array(input_masks[int(i*0.8)+int(i*0.1):]))
    test_labels.extend(np.array(labels[int(i*0.8)+int(i*0.1):]))
    data_nums.append(i+1)

with open("./data/news/社会.txt", encoding='utf-8') as f:
    input_ids, input_masks, input_types, labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
    for i, line in tqdm(enumerate(f)): 
        content = unicodedata.normalize('NFKC', line.strip())
        # encode_plus会输出一个字典，分别为'input_ids', 'token_type_ids', 'attention_mask'对应的编码
        # 根据参数会短则补齐，长则切断
        encode_dict = tokenizer.encode_plus(text=content, max_length = maxlen, 
                                            padding='max_length', truncation=True)
        input_ids.append(encode_dict['input_ids'])
        input_types.append(encode_dict['token_type_ids'])
        input_masks.append(encode_dict['attention_mask'])

        labels.append([0,0,1,0,0])

    train_input_ids.extend(np.array(input_ids[:int(i*0.8)]))
    train_input_types.extend(np.array(input_types[:int(i*0.8)]))
    train_input_masks.extend(np.array(input_masks[:int(i*0.8)]))
    train_labels.extend(np.array(labels[:int(i*0.8)]))    
    valid_input_ids.extend(np.array(input_ids[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_types.extend(np.array(input_types[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_masks.extend(np.array(input_masks[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_labels.extend(np.array(labels[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    test_input_ids.extend(np.array(input_ids[int(i*0.8)+int(i*0.1):]))
    test_input_types.extend(np.array(input_types[int(i*0.8)+int(i*0.1):]))
    test_input_masks.extend(np.array(input_masks[int(i*0.8)+int(i*0.1):]))
    test_labels.extend(np.array(labels[int(i*0.8)+int(i*0.1):]))
    data_nums.append(i+1)

with open("./data/news/科技.txt", encoding='utf-8') as f:
    input_ids, input_masks, input_types, labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
    for i, line in tqdm(enumerate(f)): 
        content = unicodedata.normalize('NFKC', line.strip())
        # encode_plus会输出一个字典，分别为'input_ids', 'token_type_ids', 'attention_mask'对应的编码
        # 根据参数会短则补齐，长则切断
        encode_dict = tokenizer.encode_plus(text=content, max_length = maxlen, 
                                            padding='max_length', truncation=True)
        input_ids.append(encode_dict['input_ids'])
        input_types.append(encode_dict['token_type_ids'])
        input_masks.append(encode_dict['attention_mask'])

        labels.append([0,0,0,1,0])
    
    train_input_ids.extend(np.array(input_ids[:int(i*0.8)]))
    train_input_types.extend(np.array(input_types[:int(i*0.8)]))
    train_input_masks.extend(np.array(input_masks[:int(i*0.8)]))
    train_labels.extend(np.array(labels[:int(i*0.8)]))    
    valid_input_ids.extend(np.array(input_ids[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_types.extend(np.array(input_types[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_masks.extend(np.array(input_masks[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_labels.extend(np.array(labels[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    test_input_ids.extend(np.array(input_ids[int(i*0.8)+int(i*0.1):]))
    test_input_types.extend(np.array(input_types[int(i*0.8)+int(i*0.1):]))
    test_input_masks.extend(np.array(input_masks[int(i*0.8)+int(i*0.1):]))
    test_labels.extend(np.array(labels[int(i*0.8)+int(i*0.1):]))
    data_nums.append(i+1)

with open("./data/news/财经.txt", encoding='utf-8') as f:
    input_ids, input_masks, input_types, labels = [], [], [] ,[] # input char ids, segment type ids,  attention mask
    for i, line in tqdm(enumerate(f)): 
        content = unicodedata.normalize('NFKC', line.strip())
        # encode_plus会输出一个字典，分别为'input_ids', 'token_type_ids', 'attention_mask'对应的编码
        # 根据参数会短则补齐，长则切断
        encode_dict = tokenizer.encode_plus(text=content, max_length = maxlen, 
                                            padding='max_length', truncation=True)
        input_ids.append(encode_dict['input_ids'])
        input_types.append(encode_dict['token_type_ids'])
        input_masks.append(encode_dict['attention_mask'])

        labels.append([0,0,0,0,1])
    
    train_input_ids.extend(np.array(input_ids[:int(i*0.8)]))
    train_input_types.extend(np.array(input_types[:int(i*0.8)]))
    train_input_masks.extend(np.array(input_masks[:int(i*0.8)]))
    train_labels.extend(np.array(labels[:int(i*0.8)]))    
    valid_input_ids.extend(np.array(input_ids[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_types.extend(np.array(input_types[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_input_masks.extend(np.array(input_masks[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    valid_labels.extend(np.array(labels[int(i*0.8):int(i*0.8)+int(i*0.1)]))
    test_input_ids.extend(np.array(input_ids[int(i*0.8)+int(i*0.1):]))
    test_input_types.extend(np.array(input_types[int(i*0.8)+int(i*0.1):]))
    test_input_masks.extend(np.array(input_masks[int(i*0.8)+int(i*0.1):]))
    test_labels.extend(np.array(labels[int(i*0.8)+int(i*0.1):]))
    data_nums.append(i+1)

print(data_nums)

10000it [00:46, 217.32it/s]
10000it [01:25, 117.38it/s]
10000it [01:28, 112.87it/s]
10000it [00:38, 262.59it/s]
10000it [01:33, 107.42it/s]


[10000, 10000, 10000, 10000, 10000]


In [14]:
test_labels.datatype

AttributeError: 'list' object has no attribute 'datatype'

In [5]:
BATCH_SIZE = 16  # 如果会出现OOM问题，减小它
# 训练集
train_data = TensorDataset(torch.LongTensor(train_input_ids), 
                           torch.LongTensor(train_input_masks), 
                           torch.LongTensor(train_input_types), 
                           torch.LongTensor(train_labels))
train_sampler = RandomSampler(train_data)  
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

# 验证集
valid_data = TensorDataset(torch.LongTensor(valid_input_ids), 
                          torch.LongTensor(valid_input_masks),
                          torch.LongTensor(valid_input_types), 
                          torch.LongTensor(valid_labels))
valid_sampler = RandomSampler(valid_data)
valid_loader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)
# 测试集（是没有标签的）
test_data = TensorDataset(torch.LongTensor(test_input_ids), 
                          torch.LongTensor(test_input_masks),
                          torch.LongTensor(test_input_types))
test_sampler = RandomSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [6]:
# 定义model
class Bert_Model(nn.Module):
    def __init__(self,classes=10):
        super(Bert_Model,self).__init__()
        self.model_name = 'hfl/chinese-roberta-wwm-ext-large'
        #self.model_name = 'hfl/chinese-roberta-wwm-ext'            #更换不同的模型
        #self.model_name = 'hfl/chinese-bert-wwm'
        self.model = BertModel.from_pretrained(self.model_name)
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.fc = nn.Linear(1024,5)     #全连接层
        self.softmax = nn.Softmax()
        
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.model(input_ids, attention_mask, token_type_ids)
        out_pool = outputs[1]   # 池化后的输出 [bs, config.hidden_size]
        out = self.fc(out_pool)   #  [bs, classes]
        out = self.softmax(out)
        return out

In [7]:
def get_parameter_number(model):
    #  打印模型参数量
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return 'Total parameters: {}, Trainable parameters: {}'.format(total_num, trainable_num)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'         #用cpu进行训练
model = Bert_Model().to(DEVICE)
print(get_parameter_number(model))

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 325527557, Trainable parameters: 325527557


In [8]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4) #AdamW优化器
EPOCHS = 5
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader),
                                            num_training_steps=EPOCHS*len(train_loader))
# 学习率先线性warmup一个epoch，然后cosine式下降。
# 加warmup（学习率从0慢慢升上去），如果把warmup去掉，可能收敛不了。

In [15]:
# 评估模型性能，在验证集上
def evaluate(model, data_loader, device):
    model.eval()
    val_true, val_pred = [], []
    with torch.no_grad():
        for idx, (ids, att, tpe, y) in (enumerate(data_loader)):
            y_pred = model(ids.to(device), att.to(device), tpe.to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
            val_true.extend(torch.argmax(y, dim=1).cpu().numpy().tolist())
    
    return accuracy_score(val_true, val_pred)  #返回accuracy


# 测试集没有标签，需要预测提交
def predict(model, data_loader, device):
    model.eval()
    val_pred1,val_pred2,val_pred3,val_pred4,val_pred5 = [],[],[],[],[]
    with torch.no_grad():
        for idx, (ids, att, tpe) in tqdm(enumerate(data_loader)):
            y_pred = model(ids.to(device), att.to(device), tpe.to(device))
            #y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            y_pred = torch.argsort(y_pred,dim=1,descending=True).detach().cpu().numpy()
            y_pred1=y_pred[:,0].tolist()
            val_pred1.extend(y_pred1)
            y_pred2=y_pred[:,1].tolist()
            val_pred2.extend(y_pred2)
            y_pred3=y_pred[:,2].tolist()
            val_pred3.extend(y_pred3)
            y_pred4=y_pred[:,3].tolist()
            val_pred4.extend(y_pred4)
            y_pred5=y_pred[:,4].tolist()
            val_pred5.extend(y_pred5)
    return val_pred1,val_pred2,val_pred3,val_pred4,val_pred5

def train_and_eval(model, train_loader, valid_loader, 
                   optimizer, scheduler, device, epoch):
    best_acc = 0.0
    patience = 0
    criterion = nn.CrossEntropyLoss()
    for i in range(epoch):
        """训练模型"""
        start = time.time()
        model.train()
        print("***** Running training epoch {} *****".format(i+1))
        train_loss_sum = 0.0
        for idx, (ids, att, tpe, y) in enumerate(train_loader):
            ids, att, tpe, y = ids.to(device), att.to(device), tpe.to(device), y.to(device)  
            y_pred = model(ids, att, tpe)
            y_real = map(float,y)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()   # 学习率变化
            
            train_loss_sum += loss.item()
            if (idx + 1) % (len(train_loader)//5) == 0:    # 只打印五次结果
            #if (idx + 1) % 5 == 0:    # 每5epoch打印结果
                print("Epoch {:04d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f}".format(
                          i+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time() - start))
                # print("Learning rate = {}".format(optimizer.state_dict()['param_groups'][0]['lr']))

        """验证模型"""
        model.eval()
        acc = evaluate(model, valid_loader, device)  # 验证模型的性能
        ## 保存最优模型
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_bert_model.pth") 
        
        print("current acc is {:.4f}, best acc is {:.4f}".format(acc, best_acc))
        print("time costed = {}s \n".format(round(time.time() - start, 5)))

In [16]:
#model.load_state_dict(torch.load("best_bert_model.pth"))
train_and_eval(model, train_loader, valid_loader, optimizer, scheduler, DEVICE, EPOCHS)

***** Running training epoch 1 *****


RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 5.78 GiB total capacity; 4.33 GiB already allocated; 27.50 MiB free; 4.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 加载最优权重对测试集测试
model.load_state_dict(torch.load("best_bert_model.pth"))
model.eval()
pred1_test,pred2_test,pred3_test,pred4_test,pred5_test = predict(model, test_loader, DEVICE)
real_test = []
for label in test_labels:
    real_test.append(test_labels.argmax())
print("\n Test Accuracy = {} \n".format(accuracy_score(real_test, pred1_test)))

top3 = (accuracy_score(real_test, pred1_test,normalize=False)+accuracy_score(real_test, pred2_test,normalize=False)+accuracy_score(real_test, pred3_test,normalize=False))/len(real_test)
print("\n Top3 Accuracy = {} \n".format(top3))
top5 = (accuracy_score(real_test, pred1_test,normalize=False)+accuracy_score(real_test, pred2_test,normalize=False)+accuracy_score(real_test, pred3_test,normalize=False)+accuracy_score(real_test, pred4_test,normalize=False)+accuracy_score(real_test, pred5_test,normalize=False))/len(real_test)
print("\n Top5 Accuracy = {} \n".format(top5))
print(classification_report(test_labels, pred1_test, digits=4))

In [ ]:
# 加载最优权重对测试集测试
model.load_state_dict(torch.load("best_bert_model.pth"))
model.eval()
sentence = 'AC米兰官方宣布将签新前锋！巨头密询范博梅尔敲定1　　新浪体育讯　在媒体三天的猜测之后，昨天，俱乐部副主席加利亚尼官方表态，亲口承认了AC米兰(微博)将在冬季引进新前锋的事实　　“我们会在冬季引进一位新前锋，”加利亚尼说，“但只是租借，因为我们要为卡萨诺保留位置。”在人选上，加利亚尼被直接问到了两个名字，这也是昨天记者在AC米兰对卡塔尼亚赛后的新闻发布会上，记者曾经询问过阿莱格里的--皮耶罗和马克西-洛佩斯　　对于皮耶罗，加利亚尼回答：“我们可不想抢劫……”而关于洛佩斯，加利亚尼的话更多一些：“他是个配得上AC米兰的球员，不过我们还有其他的候选。”之后就是关于租借和为卡萨诺保留位置的那番话，其实关于加利亚尼的话，除了一些官方性质的确认，一些问题不可太认真，近日Mediaset传出阿森纳(微博)为帕托准备了4000万的传闻，而4000万也恰恰是此前传出AC米兰为帕托的标价，如果帕托会在明夏离队，也不排除AC米兰会为今年冬天的那个前锋花费转会费　　关于AC米兰希望引进的前锋，Mediaset的巴尔吉吉亚提出的两个名字是德罗巴和博列洛，他说：“AC米兰会买进一个有实力的球员，但不是第二前锋，目前，德罗巴是第一选择，接下来是博列洛，前者一直以来都是AC米兰高层喜欢的球员，加利亚尼将试图说服切尔西(微博)放人，同时说服德罗巴相信AC米兰的计划，因此此前他已经向切尔西高层表示了自己希望离队。　　不过这次巴尔吉吉亚的观点算是“仅供参考”，因为随后《米兰新闻》就披露了在AC米兰对卡塔尼亚比赛中场休息时，布拉伊达找到范博梅尔，向他询问了马克西-洛佩斯的技术情况，并得到了荷兰人肯定的答复。之所以选择范博梅尔，是因为在05/06赛季，他和马克西在巴塞罗那曾经是队友，而如今的洛佩斯，比巴萨(微博)时代又成熟了　　由于与俱乐部关系不佳，卡塔尼亚方面对于洛佩斯去留的问题倒是开放的，昨天总经理洛-摩纳哥就公开表示：“马克西是一个优秀的球员，他配得上豪门球队，对于AC米兰他是一个不坏的选择， 但是我不清楚AC米兰的想法是什么。”洛-摩纳哥的这番表态，被认为是把洛佩斯推向AC米兰　　那么洛佩斯是否是正确的选择呢？前卡塔尼亚的中锋斯皮内西表示：“他有可能成为‘副伊布’，这是真的，他拥有强壮的身体，很好的虎丘能力，适合突前中锋的位置，他只是需要逐渐将潜力变现，相对而言，目前卡塔尼亚双前锋的打法与他的特点不符，但如果是在AC米兰，身边有罗比尼奥，则是一个合适的配置。　　但经纪人瓦尔卡雷吉的观点相反：“如果是我，我就不会买马克西-洛佩斯，我会着力培养艾尔-沙拉维，因为他已经是AC米兰的球员了。　　(沈飞'
inputs = tokenizer(sentence, return_tensors='pt')
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
pred1_test,pred2_test,pred3_test,pred4_test,pred5_test = predict(model, tokens, DEVICE)
labels = ['Sports','Entertainment','Society','Technology','Finance']
print(labels[pred1_test])